# Creating hurricane tracks using Geoanalytics

The sample code below uses big data analytics (GeoAnalytics) to reconstruct hurricane tracks using data registered on a big data file share in the GIS. Note that this functionality is currently available on ArcGIS Enterprise 10.5 and not yet with ArcGIS Online.

## Reconstruct tracks
Reconstruct tracks is a type of data aggregation tool available in the `arcgis.geoanalytics` module. This tool works with a layer of point features or polygon features that are time enabled. It first determines which points belong to a track using an identification number or identification string. Using the time at each location, the tracks are ordered sequentially and transformed into a line representing the path of movement.

## Data used
For this sample, hurricane data from over a period of 50 years, totalling about 150,000 points split into 5 shape files was used. The [National Hurricane Center](http://www.nhc.noaa.gov/gis/) provides similar datasets that can be used for exploratory purposes.

To illustrate the nature of the data a subset was published as a feature service and can be visualized as below:

In [ ]:
from arcgis.gis import GIS

# Connect to the ArcGIS Enterprise
gis = GIS("https://dev003735.esri.com/portal", "arcgis_python_api", "sharing.1")
hurricane_subset = gis.content.search("Hurricane_Subset", "Feature Layer")[0]
hurricane_subset

In [ ]:
subset_map = gis.map()
subset_map

In [ ]:
subset_map.add_layer(hurricane_subset)

## Access big data file shares
For the GeoAnalytics server to process your big data, it needs the data to be registered as a data store. In our case, the data is in multiple shape files and we will register the folder containing the files as a data store of type `bigDataFileShare`.

Get the geoanalytics datastores and search it for the registered datasets:

In [ ]:
# Query the data stores available
from arcgis import geoanalytics
datastores = geoanalytics.get_datastores()
bigdata_fileshares = datastores.search()
bigdata_fileshares

## Perform data aggregation using reconstruct tracks tool

When you add a big data file share, a corresponding item gets created in your GIS. You can search for it like a regular item and query its layers.

In [ ]:
search_result = gis.content.search("", item_type = "big data file share")
search_result

In [ ]:
data_item = search_result[0]
data_item

In [ ]:
years_50 = data_item.layers[0]
years_50

### Reconstruct tracks tool

The `reconstruct_tracks()` function is available in the `arcgis.geoanalytics.summarize_data` module. In this example, we are using this tool to aggregate the numerous points into line segments showing the tracks followed by the hurricanes. The tool creates a feature layer item as an output which can be accessed once the processing is complete.

In [ ]:
from arcgis.geoanalytics.summarize_data import reconstruct_tracks
agg_result = reconstruct_tracks(years_50, track_fields='Serial_Num',
                                method='GEODESIC')

## Inspect the results
Let us create a map and load the processed result which is a feature service

In [ ]:
processed_map = gis.map("USA")
processed_map

In [ ]:
processed_map.add_layer(agg_result)

Thus we transformed a bunch of ponints into tracks that represents paths taken by the hurricanes over a period of 50 years. We can pull up another map and inspect the results a bit more closely

Our input data and the map widget is time enabled. Thus we can filter the data to represent the tracks from only the years 1860 to 1870

In [ ]:
processed_map.set_time_extent('1860', '1870')